# Download Newsletters from The Talon Conspiracy
 

In [65]:
# Add path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
#  regular imports
from utils.selenium_resource import SeleniumResource
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
from dataclasses import dataclass
from typing import List, Optional

In [ ]:
@dataclass
class Newsletter:
    name: str
    url: str
    img_src: str
    
@dataclass
class ContentResult:
    content_title: str
    newsletters: Optional[List[Newsletter]] = None
    
@dataclass
class TTCTag:
    "The Talon Conpiracy Tags and Links"
    tag_name: str
    tag_url: str
    content: Optional[List[ContentResult]] = None

In [70]:
try:
    ttc_url = "https://thetalonconspiracy.com/"

    client = SeleniumResource()
    client.setup_for_execution()
    href_list = []
    # Navigate to the webpage
    client.driver.get(ttc_url)

    # Wait for the sidebar to be present
    sidebar = WebDriverWait(client.driver, 10).until(
        EC.presence_of_element_located((By.ID, "sidebarleft"))
    )

    # Find all anchor tags within list items in the sidebar
    # Using XPath to find all <a> tags that are descendants of <li> elements
    sidebar_tag_links = sidebar.find_elements(
        By.XPATH,
        # This finds all <a> tags with href attributes within <li> elements
        ".//li//a[@href]"
    )

    # Extract href attributes
    ttc_tags = []
    for link in sidebar_tag_links:
        tag_url = link.get_attribute('href')
        parsed_url = urlparse(tag_url).path
        tag_name = parsed_url.strip("/").split("/")[-1]

        client.driver.get(tag_url)
        content_results = WebDriverWait(client.driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CLASS_NAME, "results_content"))
        )

        ttc_tag = TTCTag(
            tag_name=tag_name,
            tag_url=tag_url,
        )

        ttc_content = []
        for result in content_results:
            result_title = result.find_element(By.TAG_NAME, "h1")
            links_with_images = result.find_elements(
                By.CSS_SELECTOR, "a:has(img)")
            
            ttc_content_result = ContentResult(
                content_title=result_title.text
            )
            ttc_newsletters = []
            for link_img in links_with_images:
                issu_link = link_img.get_attribute("href")
                issu_img_src = link_img.find_element(
                    By.TAG_NAME, "img").get_attribute("src")
                newsletter = Newsletter(
                    name = "",
                    #####
                )

    # Print results (optional)
    print(f"Found {len(href_list)} links:")
    for idx, href in enumerate(href_list, 1):
        print(f"{idx}. {href}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.teardown_after_execution()

An error occurred: Message: stale element reference: stale element not found
  (Session info: chrome=131.0.6778.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x61f04471631a <unknown>
#1 0x61f04422c6e0 <unknown>
#2 0x61f04423dfbb <unknown>
#3 0x61f04423cd45 <unknown>
#4 0x61f044232d39 <unknown>
#5 0x61f044232e65 <unknown>
#6 0x61f0442310a0 <unknown>
#7 0x61f044234c3e <unknown>
#8 0x61f0442bef6c <unknown>
#9 0x61f04429f462 <unknown>
#10 0x61f0442bded7 <unknown>
#11 0x61f04429f203 <unknown>
#12 0x61f04426dcc0 <unknown>
#13 0x61f04426ec9e <unknown>
#14 0x61f0446e3d0b <unknown>
#15 0x61f0446e7c92 <unknown>
#16 0x61f0446d0b3c <unknown>
#17 0x61f0446e8807 <unknown>
#18 0x61f0446b60df <unknown>
#19 0x61f044705578 <unknown>
#20 0x61f044705740 <unknown>
#21 0x61f044715196 <unknown>
#22 0x7bd012a591c4 <unknown>

